In [1]:
from google.colab import files
uploaded = files.upload()

!rm -rf /root/.ssh
!mkdir /root/.ssh
# !tar -xvzf ssh.tar.gz -C /root/.ssh && rm -rf ssh.tar.gz && chmod 700 /root/.ssh
!tar -xvzf ssh.tar.gz -C /root/.ssh && rm -rf ssh.tar.gz
!ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts

!git config --global user.email 'ixomaxip@gmail.com'
!git config --global user.user 'ixomaxip'
!ssh -T git@github.com

!git clone git@github.com:ixomaxip/nasoc.git

Saving ssh.tar.gz to ssh.tar.gz
./id_rsa
./id_rsa.pub
# github.com:22 SSH-2.0-babeld-968490c5
Hi ixomaxip! You've successfully authenticated, but GitHub does not provide shell access.
Cloning into 'nasoc'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 61 (delta 22), reused 12 (delta 0), pack-reused 0
Receiving objects: 100% (61/61), 407.24 KiB | 2.15 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

In [3]:
FULL_PATH = '/content/nasoc/data/en_Hasoc2021_train.csv'
TRAIN_PATH = '/content/nasoc/data/train.csv'
TEST_PATH = '/content/nasoc/data/validate.csv'

full_df = pd.read_csv(FULL_PATH)
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print(f'''shapes:
  full: {full_df.shape}
  train: {train_df.shape}
  test: {test_df.shape}
''')
print(f'''classes:
  task_1: {full_df.task_1.unique()}
  task_2: {full_df.task_2.unique()}
''')
train_df.columns

shapes:
  full: (3843, 5)
  train: (3074, 7)
  test: (769, 7)

classes:
  task_1: ['HOF' 'NOT']
  task_2: ['PRFN' 'OFFN' 'NONE' 'HATE']



Index(['Unnamed: 0', '_id', 'text', 'task_1', 'task_2', 'inputs',
       'task_2_cat'],
      dtype='object')

In [4]:
def prepare_df(path, task):
  df = pd.read_csv(path)
  df.rename(columns={f'task_{task}': 'y'}, inplace=True)
  df = df[['text', 'y']]
  return df

In [ ]:
!wget https://setup.johnsnowlabs.com/nlu/colab.sh -O - | bash 

# import nlu
import nlu-gpu

# Task 1

In [8]:
train_t1 = prepare_df(TRAIN_PATH, task=1)
test_t1 = prepare_df(TEST_PATH, task=1)
train_t1.y.unique()

array(['NOT', 'HOF'], dtype=object)

In [ ]:
trainable_pipe = nlu.load('xx.embed_sentence.labse train.sentiment')
trainable_pipe['sentiment_dl'].setMaxEpochs(60)  
trainable_pipe['sentiment_dl'].setLr(0.005)
fitted_pipe = trainable_pipe.fit(train_t1)
preds = fitted_pipe.predict(train_t1, output_level='document')
preds.dropna(inplace=True)

print(classification_report(preds['y'], preds['trained_sentiment']))
MODEL_PATH = './drive/MyDrive/nasoc/task1/sentiment_dl_baseline_trained'
fitted_pipe.save(MODEL_PATH)
preds

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
preds = fitted_pipe.predict(test_t1, output_level='document')
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['trained_sentiment']))

# Task 2

In [ ]:
train_t1 = prepare_df(TRAIN_PATH, task=2)
test_t1 = prepare_df(TEST_PATH, task=2)
test_t1.y.unique()

array(['OFFN', 'PRFN', 'HATE', 'NONE'], dtype=object)

In [ ]:
!ls ./drive/MyDrive

'Colab Notebooks'   sentiment_dl_baseline_trained
